In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
train_data = train_data = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/dermnet/train",
    batch_size=16,
    image_size=(192,192),
    shuffle=True
)

Found 15557 files belonging to 23 classes.


In [3]:
test_data = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/dermnet/test",
    batch_size=16,
    image_size=(192,192),
    shuffle=True
)

Found 4002 files belonging to 23 classes.


In [4]:


folder_names = train_data.class_names
folder_names



['Acne and Rosacea Photos',
 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions',
 'Atopic Dermatitis Photos',
 'Bullous Disease Photos',
 'Cellulitis Impetigo and other Bacterial Infections',
 'Eczema Photos',
 'Exanthems and Drug Eruptions',
 'Hair Loss Photos Alopecia and other Hair Diseases',
 'Herpes HPV and other STDs Photos',
 'Light Diseases and Disorders of Pigmentation',
 'Lupus and other Connective Tissue diseases',
 'Melanoma Skin Cancer Nevi and Moles',
 'Nail Fungus and other Nail Disease',
 'Poison Ivy Photos and other Contact Dermatitis',
 'Psoriasis pictures Lichen Planus and related diseases',
 'Scabies Lyme Disease and other Infestations and Bites',
 'Seborrheic Keratoses and other Benign Tumors',
 'Systemic Disease',
 'Tinea Ringworm Candidiasis and other Fungal Infections',
 'Urticaria Hives',
 'Vascular Tumors',
 'Vasculitis Photos',
 'Warts Molluscum and other Viral Infections']

In [5]:


# caching,shuffle and prefetching the data
train_ds = train_data.cache().shuffle(100).prefetch(buffer_size = tf.data.AUTOTUNE)
test_ds = test_data.cache().shuffle(100).prefetch(buffer_size = tf.data.AUTOTUNE)



In [6]:
# Image Preprocessing : Rescaling and Resizing
resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(192,192),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])


# Data augmentation by flipping and rotating existing images
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip(mode="horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(factor = 0.5)
])

In [7]:
# Verify the number of classes in your dataset
n_classes = len(folder_names)  # Replace with the actual number of classes in your dataset

# Build the model
resnet_model = Sequential()
pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(192,192,3),
    pooling='avg',
    classes=n_classes,
    weights='imagenet'
)

for layer in pretrained_model.layers:
    layer.trainable = False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(n_classes, activation='softmax'))  # Output layer



In [8]:
resnet_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dense_1 (Dense)             (None, 23)                11799     
                                                                 
Total params: 24,648,599
Trainable params: 1,060,887
Non-trainable params: 23,587,712
_________________________________________________________________


In [9]:
# Compile the model
resnet_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [10]:
#Fit the model with training data
resnet_model.fit(
    train_ds, epochs=20, batch_size=16, verbose=1, validation_data=test_ds
)

Epoch 1/20
973/973 [==============================] - 120s 104ms/step - loss: 2.4397 - accuracy: 0.2935 - val_loss: 2.2264 - val_accuracy: 0.3506
Epoch 2/20
973/973 [==============================] - 47s 48ms/step - loss: 2.0518 - accuracy: 0.3845 - val_loss: 2.1607 - val_accuracy: 0.3678
Epoch 3/20
973/973 [==============================] - 47s 49ms/step - loss: 1.8273 - accuracy: 0.4466 - val_loss: 2.0998 - val_accuracy: 0.3943
Epoch 4/20
973/973 [==============================] - 47s 49ms/step - loss: 1.6289 - accuracy: 0.4990 - val_loss: 2.1318 - val_accuracy: 0.3998
Epoch 5/20
973/973 [==============================] - 48s 49ms/step - loss: 1.4423 - accuracy: 0.5522 - val_loss: 2.1104 - val_accuracy: 0.4085
Epoch 6/20
973/973 [==============================] - 48s 50ms/step - loss: 1.2645 - accuracy: 0.6032 - val_loss: 2.1620 - val_accuracy: 0.4155
Epoch 7/20
973/973 [==============================] - 49s 50ms/step - loss: 1.1054 - accuracy: 0.6488 - val_loss: 2.2054 - val_accurac

In [12]:
resnet_model.save("derma.h5") 